In [3]:
from bhpwave.trajectory.inspiral import IntegrateInspiralGeneratorBase
from bhpwave.trajectory.inspiral import TrajectoryData, InspiralGenerator
from bhpwave.waveform import KerrWaveform
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from bhpwave.trajectory.inspiral import spin_of_chi, omega_of_a_alpha, pn_flux_noprefactor
from bhpwave.trajectory.geodesic import kerr_circ_geo_radius, kerr_isco_frequency, kerr_circ_geo_orbital_frequency

In [5]:
traj = TrajectoryData()

In [6]:
def Edot_test(chi, alpha, R):
    a = spin_of_chi(chi)
    omega = omega_of_a_alpha(a, alpha)
    r0 = kerr_circ_geo_radius(a, omega)
    if traj.range_check(a, omega):
        return traj.scaled_energy_flux(a, r0)/pn_flux_noprefactor(omega)*(1 - 0.5*R**2)
    else:
        return 0.

In [7]:
test_gen = IntegrateInspiralGeneratorBase(Edot_test)

In [8]:
kerr_gen_mod = KerrWaveform(trajectory_data=test_gen)

In [9]:
kerr_gen_GR = KerrWaveform()

In [10]:
M = 1e6  # primary mass in solar masses
mu = 3e1 # secondary mass in solar masses
a = 0.9 # dimensionless spin of the primary
p0 = 6.55 # initial semi-latus rectum
e0 = 0.0 # eccentricity is ignored for circular orbits
x0 = 1.0  # inclination is ignored for circular orbits
qK = 0.8  # polar angle of Kerr spin angular momentum
phiK = 0.2  # azimuthal angle of Kerr spin angular momentum
qS = 0.3  # polar sky angle
phiS = 0.3  # azimuthal sky angle
dist = 2.0  # distance to source in Gpc
Phi_phi0 = 0.2 # initial azimuthal position of the secondary
Phi_theta0 = 0.0 # ignored for circular orbits
Phi_r0 = 0.0 # ignored for circular orbits
dt = 5.0  # time steps in seconds
T = 1.0  # waveform duration in years

injection_paramters = [M, mu, a, p0, e0, x0, 
                       dist, qS, phiS, qK, phiK, 
                       Phi_phi0, Phi_theta0, Phi_r0, 
                       dt, T]

In [11]:
R = 5e-3
h = kerr_gen_mod(*injection_paramters, R)
h_GR = kerr_gen_GR(*injection_paramters)

In [ ]:
plt.plot(h.real[-1000:])
plt.plot(h_GR.real[-1000:])
plt.show()